# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

[In this notebook you're provided with hints and brief instructions and thinking prompts. Don't ignore them as they are designed to equip you with the structure for the project and will help you analyze what you're doing on a deeper level. Before submitting your project, make sure you remove all hints and descriptions provided to you. Instead, make this report look as if you're sending it to your teammates to demonstrate your findings - they shouldn't know you had some external help from us! To help you out, we've placed the hints you should remove in square brackets.]

[Before you dive into analyzing your data, explain the purposes of the project and hypotheses you're going to test.]

## Open the data file and have a look at the general information. 

[Start with importing the libraries and loading the data. You may realise that you need additional libraries as you go, which is totally fine - just make sure to update this section when you do.]

In [1]:
# Loading all the libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
try:
    df = pd.read_csv("/datasets/credit_scoring_eng.csv")
except:
    df = pd.read_csv("D:\My Dokuments\אישי\Practicum100\credit_scoring_eng.csv")
# Load the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.describe()
print()
#df.describe(include = "all").T
df.describe(include = object)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,secondary education,married,F,employee,wedding ceremony
freq,13750,12380,14236,11119,797


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

[Now let's explore our data. You'll want to see how many columns and rows it has, look at a few rows to check for potential issues with the data.]

In [3]:
# Let's see how many rows and columns our dataset has
print("the size of the table is {} rows and {} columns".format(df.shape[0],df.shape[1]))


the size of the table is 21525 rows and 12 columns


In [4]:
# let's print the first N rows
df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


[Describe what you see and notice in your printed data sample. Are there any issues that may need further investigation and changes?]

In [5]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Are there missing values across all columns or just a few? Briefly describe what you see in 1-2 sentences.]
- I notice there are 2 columns with float64 dtype, so there should be missing data
- Also I see in those 2 columns that the number of rows are less than the total rows with difference of: 21525 - 19351 = 2174

In [6]:
print("misiing data in days_employed column: ",df["days_employed"].isnull().sum()) 

misiing data in days_employed column:  2174


In [7]:
print("total number of rows:",len(df.index)) 

total number of rows: 21525


In [8]:
# Let's look at the filtered table with missing values in the first column with missing data
print("number of unmissing value rows in days_employed:",df["days_employed"].count())


number of unmissing value rows in days_employed: 19351


In [9]:
print("misiing data in total_income columns:",df["total_income"].isnull().sum())

misiing data in total_income columns: 2174


[Do missing values seem symmetric? Can we be sure in this assumption? Explain your thoughts briefly in this section. You may probably want to conduct further investigations, and count the missing values in all the rows with missing values to confirm the missing samples are of the same size.]

As I metioned above

In [10]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
df[(df["days_employed"].isnull() & df["total_income"].isnull())].count() # misiing data (NaN) in both columns


children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

# **Intermediate conclusion**

[Does the number of rows in the filtered table match the number of missing values? What conclusion can we make from this?]

<span style="color:blue; font-size: 18px">**yes it is the same number that means all the missing values are symmetric**</span>

[Calculate the percentage of the missing values compared to the whole dataset. Is it a considerably large piece of data? If so, you may want to fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. You will need to decide which characteristic *you* think might be the reason. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.]

[Explain your next steps and how they correlate with the conclusions you made so far.]

In [11]:
print("percentage of the missing values compared to the whole dataset is {:.0%}".format(df["days_employed"].isnull().sum() / len(df.index)))

percentage of the missing values compared to the whole dataset is 10%


<span style="color:blue; font-size: 18px">**the percentage is considerably large piece of data, so we can not delete them**</span>

In [12]:
df["days_employed"].isnull().value_counts()

False    19351
True      2174
Name: days_employed, dtype: int64

In [13]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
df["total_income"].median()
print()
df["total_income"].mean()

23202.87

26787.56835465867

In [14]:
# all unique values those not by missing values

In [15]:
print("all unique values those not by missing values:")
df[(df.notna())].nunique()

all unique values those not by missing values:


children                8
days_employed       19351
dob_years              58
education              15
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19348
purpose                38
dtype: int64

# [Describe your findings here.]

**Possible reasons for missing values in data**

<span style="color:blue; font-size: 18px">**missing values in total_income are distributed evenly among all employment types**</span>.
            
            
[Propose your ideas on why you think the values might be missing. Do you think they are missing randomly or there are any patterns?]
[Let's start checking whether the missing values are random.]

**Intermediate conclusion**

[Is the distribution in the original dataset similar to the distribution of the filtered table? What does that mean for us?]

<span style="color:blue; font-size: 18px">**we can't make any conclusions yet**</span>.

[If you think we can't make any conclusions yet, let's investigate our dataset further. Let's think about other reasons that could lead to data missing and check if we can find any patterns that may lead us to thinking that the missing values are not random. Because this is your work, this section is optional.]

 <span style="color:blue; font-size: 18px">**they got lost!!**</span>.

<span style="color:blue; font-size: 18px">**for education we can index it according to it`s id column, the same with family_status**</span>.

**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.]

In [16]:
# Checking for other patterns - explain which

In [17]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [18]:
df1=df[["days_employed","total_income"]]
df1[(df1.days_employed.isnull())].nunique()

days_employed    0
total_income     0
dtype: int64

In [19]:
df.education[(df.education_id==1)].unique()

array(['secondary education', 'Secondary Education',
       'SECONDARY EDUCATION'], dtype=object)

In [20]:
df.children.unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

<span style="color:red; font-size: 22px">**1-children**
    
<span style="color:blue; font-size: 18px">**Here we can notice that 0 and 20 are error values in the number of the children!**  
    **that we have to fix it**

In [21]:
df.days_employed.unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

<span style="color:red; font-size: 22px">**2-days_employed**

<span style="color:blue; font-size: 18px">**here we have to deal with negative values! and maybe convert the column in date form**

In [22]:
df.dob_years.value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

<span style="color:red; font-size: 22px">**3-dob_years**

<span style="color:blue; font-size: 18px">**here we have to deal with the 0 and to categorize the different age ranges**

In [23]:
df.education.unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

<span style="color:red; font-size: 22px">**4-education**

<span style="color:blue; font-size: 18px">**here we have to form the text in lowercase**

In [24]:
df.education_id.unique()

array([0, 1, 2, 3, 4])

<span style="color:red; font-size: 22px">**5-education_id**

<span style="color:blue; font-size: 18px">**here is nothing to do**

In [25]:
df.family_status.unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

<span style="color:red; font-size: 22px">**6-family_status**

<span style="color:blue; font-size: 18px">**here we notice that the widow and the divorced are identical to unmarried, so I will change those 2 definitions in unmarried**

In [26]:
df.family_status_id.unique()

array([0, 1, 2, 3, 4])

<span style="color:red; font-size: 22px">**7-family_status_id**

<span style="color:blue; font-size: 18px">**here we notice that the widow and the divorced are identical to unmarried, so I will change theose 2 definitions in unmarried**

<span style="color:red; font-size: 22px">**7-family_status_id**

<span style="color:blue; font-size: 18px">**here is nothing to do**

In [27]:
df.gender.unique()

array(['F', 'M', 'XNA'], dtype=object)

<span style="color:red; font-size: 22px">**8-gender**

<span style="color:blue; font-size: 18px">**here I have to check the XNA**

In [28]:
df.income_type.unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

<span style="color:red; font-size: 22px">**9-income_type**

<span style="color:blue; font-size: 18px">**here I have to categorize 2 groups  
    empolyed ('employee','business','civil servant','entrepreneur')  
    and  
    unemployed ('retiree','unemployed','student','paternity / maternity leave')**

In [29]:
df.debt.unique()

array([0, 1])

<span style="color:red; font-size: 22px">**10-debt**

<span style="color:blue; font-size: 18px">**here is nothing to do**

In [30]:
df.total_income.describe(include="all")

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

<span style="color:red; font-size: 22px">**11-total_income**

<span style="color:blue; font-size: 18px">**here I have to replace missing values with mean or median value according to different categories**

In [31]:
df.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

<span style="color:red; font-size: 22px">**12-purpose**

<span style="color:blue; font-size: 18px">**here I have to make Stemming and lemmatization**

## Data transformation

[Let's go through each column to see what issues we may have in them.]

[Begin with removing duplicates and fixing educational information if required.]

<span style="color:red; font-size: 22px">**4-education**

<span style="color:blue; font-size: 18px">**here we have to form the text in lowercase**

In [32]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df["education"].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

<span style="color:blue; font-size: 18px">**since I noticed that education column includes upper and lower case in text, I`ll make them all in lower case**</span>.

In [33]:
# Fix the registers if required
df["education"] = df["education"].str.lower() # making lowcase of the "education" column
df["education"].unique()
df["education"].value_counts()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

[Check the data the `children` column]

<span style="color:red; font-size: 22px">**1-children**
    
<span style="color:blue; font-size: 18px">**Here we can notice that 0 and 20 are error values in the number of the children!**  
    **that we have to fix it**

In [34]:
# Let's see the distribution of values in the `children` column
df["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [35]:
df["children"].value_counts().sum()

21525

In [36]:
len(df[(df["children"]==-1)])

47

In [37]:
"{:.2%}".format(len(df[(df["children"]==-1)])/df["children"].value_counts().sum())

'0.22%'

[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.]

    0.22% is not high percentage of problematic data, and it can not be in minus, so i suggest to eleminate the minus sign

In [38]:
# [fix the data based on your decision]
df["children"]=df["children"].replace({-1: 1}) # replacing -1 values
df["children"]=df["children"].replace({20: 2})
df["children"].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [39]:
# Checking the `children` column again to make sure it's all fixed
len(df[(df["children"]==-1)])
len(df[(df["children"]==20)])

0

0

[Check the data in the `days_employed` column. Firstly think about what kind of issues could there be and what you may want to check and how you will do it.]

<span style="color:red; font-size: 22px">**2-days_employed**

<span style="color:blue; font-size: 18px">**here we have to deal with negative values! and maybe convert the column in date form**

In [40]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
df["days_employed"].isnull().value_counts()

False    19351
True      2174
Name: days_employed, dtype: int64

In [41]:
"The percentage of days_employed {:.0%}".format(df["days_employed"].isnull().sum() / len(df.index))

'The percentage of days_employed 10%'

[If the amount of problematic data is high, it could've been due to some technical issues. We may probably want to propose the most obvious reason why it could've happened and what the correct data might've been, as we can't drop these problematic rows.]

In [42]:
# Address the problematic values, if they exist
len(df["days_employed"][(df["days_employed"] < 0)]) # rows with negative values

15906

In [43]:
len(df["days_employed"])

21525

In [44]:
df["days_employed"].isnull().value_counts()

False    19351
True      2174
Name: days_employed, dtype: int64

In [45]:
df_days_employed=df.days_employed[(df.days_employed.isnull())]
df_days_employed

12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: days_employed, Length: 2174, dtype: float64

In [46]:
# Check the result - make sure it's fixed
df.days_employed.isnull().value_counts()
df.days_employed.head()

False    19351
True      2174
Name: days_employed, dtype: int64

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [47]:
df.days_employed[(df.days_employed<0)].count()
df.days_employed=df.days_employed[(df.days_employed.notna())].astype("float").abs()
df.days_employed[(df.days_employed<0)].count()

15906

0

<span style="color:blue; font-size: 18px">**here I`ve changed negative values and kept the missing values**

<span style="color:red; font-size: 22px">**3-dob_years**

<span style="color:blue; font-size: 18px">**here we have to deal with the 0**

[Let's now look at the client's age and whether there are any issues there. Again, think about what can data can be strange in this column, i.e. what cannot be someone's age.]

<span style="color:blue; font-size: 18px">age 0 is unthinkable!!

In [48]:
# Check the `dob_years` for suspicious values and count the percentage
df["dob_years"].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [49]:
A=len(df[(df["dob_years"]==0)])/len(df["dob_years"])
A
print("the percentage is: {:.2%} ".format(A))
# count the percentage

0.004692218350754936

the percentage is: 0.47% 


[Decide what you'll do with the problematic values and explain why.]

In [50]:
# Address the issues in the `dob_years` column, if they exist
df["dob_years"].median()

42.0

<span style="color:blue; font-size: 18px">**here I am changing the 0 value into the median of dob_years because the percentage is low**

In [51]:
df["dob_years"]=df["dob_years"].replace({0: df["dob_years"].median()}) # replacing 0 values with median


<span style="color:blue; font-size: 18px">**here I am changing the type values**

In [52]:
# Check the result - make sure it's fixed
df["dob_years"].astype("int").value_counts().sort_index() # fixed

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    698
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]

<span style="color:blue; font-size: 18px">**here I am changing the 0 value into the median of dob_years**

<span style="color:red; font-size: 22px">**6-family_status**

<span style="color:blue; font-size: 18px">**here we notice that the widow and the divorced are identical to unmarried, so I will change those 2 definitions in unmarried**

In [53]:
# Let's see the values for the column
df["family_status"].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [54]:
married=df.family_status[(df.family_status=="married")].count()/(df.family_status.count())
civil=df.family_status[(df.family_status=="civil partnership")].count()/(df.family_status.count())
unmarried=df.family_status[(df.family_status=="unmarried")].count()/(df.family_status.count())
print("married: {:.1%}".format(married))
print("civil: {:.1%}".format(civil))
print("unmarried: {:.1%}".format(unmarried))

married: 57.5%
civil: 19.4%
unmarried: 13.1%


In [55]:
df.family_status.count()

21525

In [56]:
# Address the problematic values in `family_status`, if they exist
df["family_status"]=df["family_status"]
df.loc[(df.family_status == 'divorced'),'family_status']='unmarried'
df.loc[(df.family_status == 'widow / widower'),'family_status']='unmarried'

In [57]:
# Check the result - make sure it's fixed
df.family_status.value_counts()

married              12380
unmarried             4968
civil partnership     4177
Name: family_status, dtype: int64

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

<span style="color:red; font-size: 22px">**8-gender**

<span style="color:blue; font-size: 18px">**here I have to check the XNA**

<span style="color:red; font-size: 22px">**6-family_status**

<span style="color:blue; font-size: 18px">**here we notice that the widow and the divorced are identical to unmarried, so I will change those 2 definitions in unmarried**

In [58]:
df["gender"].value_counts() # Let's see the values in the column

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [59]:
# Address the problematic values, if they exist

<span style="color:blue; font-size: 18px">**XNA undefined gender and just one count of it, so it can be deleted**

In [60]:
# Check the result - make sure it's fixed
df=df[(df["gender"]!="XNA")] # deleting the row where "gender"=XNA 
df["gender"].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]

<span style="color:red; font-size: 22px">**9-income_type**

<span style="color:blue; font-size: 18px">**here I have to categorize 2 groups  
    empolyee ('employee','business','civil servant','entrepreneur')  
    and  
    unemployed ('retiree','unemployed','student','paternity / maternity leave')  
    and  
    business ('business','entrepreneur')**

<span style="color:blue; font-size: 18px">**XNA undefined gender and just one count of it, so it can be deleted**

In [61]:
# Address the problematic values, if they exist

In [62]:
df["income_type"].value_counts()# Let's see the values in the column

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [63]:
df.loc[(df.income_type == 'civil servant'),'income_type']='employee'
df.loc[(df.income_type == 'entrepreneur'),'income_type']='business'
df.loc[(df.income_type == 'retiree'),'income_type']='unemployed'
df.loc[(df.income_type == 'student'),'income_type']='unemployed'
df.loc[(df.income_type == 'paternity / maternity leave'),'income_type']='unemployed'

In [64]:
employee=df.income_type[(df.income_type=="employee")].count()/df.income_type.count()
business =df.income_type[(df.income_type=="business")].count()/df.income_type.count()
unemployed=df.income_type[(df.income_type=="unemployed")].count()/df.income_type.count()
print("employee: {:.1%}".format(employee))
print("business: {:.1%}".format(business))
print("unemployed: {:.1%}".format(unemployed))

employee: 58.4%
business: 23.6%
unemployed: 17.9%


In [65]:
# Check the result - make sure it's fixed
df["income_type"].value_counts()

employee      12578
business       5086
unemployed     3860
Name: income_type, dtype: int64

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

<span style="color:red; font-size: 22px">**Duplicates**

In [66]:
# Checking duplicates
df.duplicated().value_counts()


False    21449
True        75
dtype: int64

In [67]:
df.duplicated().sum() # number of the duplicated rows in whole table

75

In [68]:
# Address the duplicates, if they exist
df=df.drop_duplicates()

In [69]:
df["dob_years"].isnull().value_counts()

False    21449
Name: dob_years, dtype: int64

<span style="color:blue; font-size: 18px">**there are 75 rows duplicated cleaned**

In [70]:
# Last check whether we have any duplicates
df.duplicated().sum() 

0

In [71]:
# Check the size of the dataset that you now have after your first manipulations with it

In [72]:
df.info()
#df.describe(include="all").T
df.describe(include="object")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 12 columns):
children            21449 non-null int64
days_employed       19350 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        19350 non-null float64
purpose             21449 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


,education,family_status,gender,income_type,purpose
count,21449,21449,21449,21449,21449
unique,5,3,2,3,38
top,secondary education,married,F,employee,wedding ceremony
freq,15170,12336,14171,12537,791


[Describe your new dataset: briefly say what's changed and what's the percentage of the changes, if there were any.]


<span style="color:blue; font-size: 18px">**-the number of all rows unless the rows of the unmissing values in the 2 column  
    -the education column categories has changed from 15 to 5 because the text form  
    -the family_status column categories has changed from 5 to 3  
    -the income_type column categories has changed from 8 to 3**

In [73]:
df.family_status.unique()

array(['married', 'civil partnership', 'unmarried'], dtype=object)

In [74]:
married=df.family_status[(df.family_status=="married")].count()/(df.family_status.count())
civil=df.family_status[(df.family_status=="civil partnership")].count()/(df.family_status.count())
unmarried=df.family_status[(df.family_status=="unmarried")].count()/(df.family_status.count())
print("married: {:.1%}".format(married))
print("civil: {:.1%}".format(civil))
print("unmarried: {:.1%}".format(unmarried))

married: 57.5%
civil: 19.3%
unmarried: 23.1%


<span style="color:blue; font-size: 18px">**percentage of the changes in family_status  
        -unmarried from 13.1% to 23.1%**

In [75]:
employee=df.income_type[(df.income_type=="employee")].count()/df.income_type.count()
business =df.income_type[(df.income_type=="business")].count()/df.income_type.count()
unemployed=df.income_type[(df.income_type=="unemployed")].count()/df.income_type.count()
print("employee: {:.1%}".format(employee))
print("business: {:.1%}".format(business))
print("unemployed: {:.1%}".format(unemployed))

employee: 58.5%
business: 23.7%
unemployed: 17.9%


<span style="color:blue; font-size: 18px">**percentages of the changes in income_type has been a very little changed**

# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]

<span style="color:blue; font-size: 18px">**-the number of all rows unless the rows of the unmissing values in the 2 column  
    -the education column categories has changed from 15 to 5  
    -the family_status column categories has changed from 5 to 3  
    -the income_type column categories has changed from 8 to 3**

In [76]:
# Find the dictionaries

<span style="color:blue; font-size: 18px">**education  
    and  
    Family_status  
    every column has it`s ID column**

### Restoring missing values in `total_income`

[Briefly state which column(s) have values missing that you need to address. Explain how you will fix them.]


[Start with addressing total income missing values. Create an age category for clients. Create a new column with the age category. This strategy can help with calculating values for the total income.]


<span style="color:red; font-size: 22px">**adding age_group as a new column**

In [77]:
# Let's write a function that calculates the age category
def age_group(age):
    if age <= 18:
        return 'children'
    if age <= 62:
        return 'adult'
    return 'retired' 

In [78]:
# Test if the function works
print("the age of 65 is defined as {}: ".format(age_group(65)))

the age of 65 is defined as retired: 


In [79]:
# Creating new column based on function
df["age_group"]=df['dob_years'].apply(age_group)

In [80]:
# Checking how values in the new column

df["age_group"].value_counts()
df["age_group"].describe()

adult      20025
retired     1424
Name: age_group, dtype: int64

count     21449
unique        2
top       adult
freq      20025
Name: age_group, dtype: object

[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]

[Create a table that only has data without missing values. This data will be used to restore the missing values.]

<span style="color:blue; font-size: 22px">**table that only has data without missing values**

In [81]:
df.groupby(["income_type","age_group"]).total_income.agg(["count","mean"])

count          mean
income_type age_group                     
business    adult       4503  32385.094364
            retired       74  33128.604608
employee    adult      11186  25973.933271
            retired      140  27860.458957
unemployed  adult       2375  22290.795181
            retired     1072  21144.116941

In [82]:
df["income_type"].unique()

array(['employee', 'unemployed', 'business'], dtype=object)

<span style="color:blue; font-size: 22px">**list of the averages of total_income by every income_type**

In [83]:
#df[df.income_type=="business"].total_income.mean()  # the average of total_income when income_type=="business"
df.groupby("income_type")["total_income"].describe()

,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
business,4577.0,32397.115286,20888.767605,4592.450,20142.0390,27577.272,39027.4220,362496.645
employee,11326.0,25997.252501,14724.691300,3418.824,16496.6835,22927.388,31712.8265,276204.162
unemployed,3447.0,21934.183904,12837.472325,3306.762,13259.3865,18956.934,27152.0690,117616.523


In [84]:
df.total_income[(df.income_type=="employee")].isnull().value_counts()

False    11326
True      1211
Name: total_income, dtype: int64

In [85]:
#  Write a function that we will use for filling in missing values

<span style="color:blue; font-size: 22px">**filling all missing values in total_income with the mean value of each category in income_type**

In [86]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda grp: grp.fillna(np.median(grp)))
#df['total_income'] = df.groupby('income_type')['total_income'].transform('median'))

In [87]:
#Reviewer's code
print("Code with implementing data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda grp: grp.fillna(grp.median()))")
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda grp: grp.fillna(np.median(grp)))
display(df[df['total_income'].isnull()])
print('Number of Nan values:', len(df[df['total_income'].isnull()]))
print('-------------------------------------------------------------------------------')
print("Code with implementing  df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))")
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))
display(df[df['total_income'].isnull()])
print('Number of Nan values:', len(df[df['total_income'].isnull()]))

Code with implementing data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda grp: grp.fillna(grp.median()))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65.0,secondary education,1,civil partnership,1,M,unemployed,0,NaN,to have a wedding,retired
26,0,NaN,41.0,secondary education,1,married,0,M,employee,0,NaN,education,adult
29,0,NaN,63.0,secondary education,1,unmarried,4,F,unemployed,0,NaN,building a real estate,retired
41,0,NaN,50.0,secondary education,1,married,0,F,employee,0,NaN,second-hand car purchase,adult
55,0,NaN,54.0,secondary education,1,civil partnership,1,F,unemployed,1,NaN,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47.0,secondary education,1,married,0,M,business,0,NaN,purchase of a car,adult
21495,1,NaN,50.0,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,adult
21497,0,NaN,48.0,bachelor's degree,0,married,0,F,business,0,NaN,building a property,adult
21502,1,NaN,42.0,secondary education,1,married,0,F,employee,0,NaN,building a real estate,adult


Number of Nan values: 2099
-------------------------------------------------------------------------------
Code with implementing  df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


Number of Nan values: 0


In [88]:
# Check if it works


In [89]:
df.total_income.isnull().value_counts()
df.info()

False    21449
Name: total_income, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 13 columns):
children            21449 non-null int64
days_employed       19350 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null float64
purpose             21449 non-null object
age_group           21449 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.9+ MB


In [90]:
# Apply it to every row


### Check if we got any errors

In [91]:
#df.groupby('income_type')['total_income'].describe(include="all").T
df.groupby('income_type')['total_income'].median()

income_type
business      27577.272
employee      22927.388
unemployed    18956.934
Name: total_income, dtype: float64

In [92]:
# Look at the median values for income based on your identified factors

[Repeat such comparisons for multiple factors. Make sure you consider different aspects and explain your thinking process.]



[Make a decision on what characteristics define income most and whether you will use a median or a mean. Explain why you made this decision]


[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]


In [93]:
# Replacing missing values if there are any errors
df.isnull().sum() # it stays the problem with the problematic column!

children               0
days_employed       2099
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
dtype: int64

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [94]:
# Checking the number of entries in the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 13 columns):
children            21449 non-null int64
days_employed       19350 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null float64
purpose             21449 non-null object
age_group           21449 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.9+ MB


###  Restoring values in `days_employed`

[Think about the parameters that may help you restore the missing values in this column. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. You will probably conduct a research similar to the one you've done when restoring data in a previous column.]

<span style="color:red; font-size: 22px">**2-days_employed**

<span style="color:blue; font-size: 18px">**continue checking**

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 13 columns):
children            21449 non-null int64
days_employed       19350 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null float64
purpose             21449 non-null object
age_group           21449 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.9+ MB


In [96]:
# Distribution of `days_employed` medians based on your identified parameters

<span style="color:blue; font-size: 18px">**Assuming that the maximum days_employed is possible based on age**

In [97]:
df.days_employed.mean()
df.days_employed.median()

66918.06514084844

2194.218767670908

In [98]:
df.days_employed[(df.days_employed.notna())].count()

19350

In [99]:
df.days_employed[(df.age_group=="adult")].mean()
df.days_employed[(df.age_group=="adult")].median()

49930.5650171818

1984.8718839314465

In [100]:
df.days_employed[(df.age_group=="retired")].mean()
df.days_employed[(df.age_group=="retired")].median()

305535.6407504242

359016.21144463355

<span style="color:blue; font-size: 18px">**adding new column for max_days according the age**

In [101]:
def max_days(age):
    A=0
    A=1000+((age-18)*257)
    return A

In [102]:
df["max_days"]=df["dob_years"].apply(max_days)

In [103]:
df.days_employed[1]

4024.803753850451

In [104]:
df.max_days.count()
df.max_days.head()

21449

0    7168.0
1    5626.0
2    4855.0
3    4598.0
4    9995.0
Name: max_days, dtype: float64

<span style="color:blue; font-size: 18px">**filling missing with 0**

In [105]:
df.days_employed.fillna(0,inplace=True)
df.days_employed.isnull().value_counts()


False    21449
Name: days_employed, dtype: int64

<span style="color:blue; font-size: 18px">**mean or median days of every income_type**

In [106]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter


In [107]:
business=df.days_employed[(df.income_type=="business")].median()
print("business median =",business)
unemployed=df.days_employed[(df.income_type=="unemployed")].median()
print("unemployed median =",unemployed)
employee=df.days_employed[(df.income_type=="employee")].median()
print("employee median =",employee)
print("business mean =",df.days_employed[(df.income_type=="business")].mean())
print("unemployed mean =",df.days_employed[(df.income_type=="unemployed")].mean())
print("employee mean =",df.days_employed[(df.income_type=="employee")].mean())

business median = 1311.46487211657
unemployed median = 360761.97481084993
employee median = 1449.2485516117881
business mean = 1902.462968810784
unemployed mean = 328057.32407332666
employee mean = 2214.1044814362567


<span style="color:blue; font-size: 18px">**median days of every income_type**

In [108]:
# Apply function to the income_type

In [109]:
df.income_type.unique()

array(['employee', 'unemployed', 'business'], dtype=object)

In [110]:
# Check that the function works

In [111]:
df[["days_employed","max_days","dob_years"]].head()

,days_employed,max_days,dob_years
0,8437.673028,7168.0,42.0
1,4024.803754,5626.0,36.0
2,5623.422610,4855.0,33.0
3,4124.747207,4598.0,32.0
4,340266.072047,9995.0,53.0


In [112]:
import numpy as np
df["days_employed"] = np.where(df["days_employed"] > df["max_days"], df["max_days"],df["days_employed"])

In [113]:
df[["days_employed","max_days","dob_years"]].head()

,days_employed,max_days,dob_years
0,7168.000000,7168.0,42.0
1,4024.803754,5626.0,36.0
2,4855.000000,4855.0,33.0
3,4124.747207,4598.0,32.0
4,9995.000000,9995.0,53.0


In [114]:
df.days_employed.loc[df["days_employed"]==0].head()


12    0.0
26    0.0
29    0.0
41    0.0
55    0.0
Name: days_employed, dtype: float64

In [115]:
df.days_employed.isnull().value_counts()

False    21449
Name: days_employed, dtype: int64

In [116]:
df["days_employed"].replace(0,np.nan,True)
df.days_employed.isnull().value_counts()

False    19350
True      2099
Name: days_employed, dtype: int64

In [117]:
df.days_employed.loc[(df.days_employed.isnull()) & (df.income_type=="employee")] = employee
df.days_employed.loc[(df.days_employed.isnull()) & (df.income_type=="unemployed")] = unemployed
df.days_employed.loc[(df.days_employed.isnull()) & (df.income_type=="business")] = business

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<span style="color:blue; font-size: 18px">**so we may just fill the missing values with the mean value of the converted dates**

In [118]:
# Distribution of `days_employed` means based on your identified parameters

In [119]:
df.days_employed[df.income_type=="employee"].mean()
df.days_employed[df.income_type=="unemployed"].mean()
df.days_employed[df.income_type=="business"].mean()

2293.9407153084217

46777.190396165824

2000.794150091819

[Decide what you will use: means or medians. Explain why.]

<span style="color:blue; font-size: 18px">**I am not sure which is better!**

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [120]:
# Check the entries in all columns - make sure we fixed all missing values

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 14 columns):
children            21449 non-null int64
days_employed       21449 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null float64
purpose             21449 non-null object
age_group           21449 non-null object
max_days            21449 non-null float64
dtypes: float64(4), int64(4), object(6)
memory usage: 3.1+ MB


In [122]:
# Check if function worked

In [123]:
df.days_employed.isnull().value_counts()

False    21449
Name: days_employed, dtype: int64

In [124]:
# Replacing missing values

<span style="color:green; font-size: 22px">**After correcting the days_employed column and filling the missing values and the corrected one, which had values exceeded the max days of what a person can work in his life, it gave the possibility to categorize it according to the income_type.**

## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]

[Despite of how you decide to address the categorization, make sure to provide clear explanation of why you made your decision. Remember: this is your work and you make all decisions in it.]


<span style="color:red; font-size: 22px">**12-purpose**

<span style="color:blue; font-size: 18px">**here I have to make Stemming and lemmatization**

In [125]:
df.purpose.unique()# Check the unique values

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [126]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [127]:
# Print the values for your selected data for categorization
import nltk
from nltk.stem import WordNetLemmatizer

wordnet_lemma = WordNetLemmatizer()

[Let's check unique values]

[What main groups can you identify based on the unique values?]

[Based on these themes, we will probably want to categorize our data.]


In [128]:
# Let's write a function to categorize the data based on common topics
lemmas_list_all = []

for purpose in df.purpose.unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = "n") for w in words]
    lemmas=[l.lower() for l in lemmas]
    for i in lemmas:
        if i in lemmas_list_all:
            continue
        else:
            lemmas_list_all.append(i)

In [129]:
lemmas_list_all

['purchase',
 'of',
 'the',
 'house',
 'car',
 'supplementary',
 'education',
 'to',
 'have',
 'a',
 'wedding',
 'housing',
 'transaction',
 'having',
 'for',
 'my',
 'family',
 'buy',
 'real',
 'estate',
 'commercial',
 'residential',
 'construction',
 'own',
 'property',
 'building',
 'buying',
 'second-hand',
 'with',
 'become',
 'educated',
 'getting',
 'an',
 'ceremony',
 'get',
 'higher',
 'profile',
 'university',
 'renting',
 'out',
 'renovation',
 'going']

In [130]:
# Create a column with the categories and count the values for them
Counter(lemmas_list_all)


Counter({'purchase': 1,
         'of': 1,
         'the': 1,
         'house': 1,
         'car': 1,
         'supplementary': 1,
         'education': 1,
         'to': 1,
         'have': 1,
         'a': 1,
         'wedding': 1,
         'housing': 1,
         'transaction': 1,
         'having': 1,
         'for': 1,
         'my': 1,
         'family': 1,
         'buy': 1,
         'real': 1,
         'estate': 1,
         'commercial': 1,
         'residential': 1,
         'construction': 1,
         'own': 1,
         'property': 1,
         'building': 1,
         'buying': 1,
         'second-hand': 1,
         'with': 1,
         'become': 1,
         'educated': 1,
         'getting': 1,
         'an': 1,
         'ceremony': 1,
         'get': 1,
         'higher': 1,
         'profile': 1,
         'university': 1,
         'renting': 1,
         'out': 1,
         'renovation': 1,
         'going': 1})

[If you decide to categorize the numerical data, you'll need to come up with the categories for it too.]

In [131]:
# Looking through all the numerical data in your selected column for categorization


<span style="color:blue; font-size: 18px">**I have no idea what it talks about!**

In [132]:
# Getting summary statistics for the column



<span style="color:blue; font-size: 18px">**I don`t know what to write!**

[Decide what ranges you will use for grouping and explain why.]

In [133]:
purchase_category = ['purchase of the house','car purchase','purchase of the house for my family',
                     'buying a second-hand car','buying my own car','cars','car','second-hand car purchase',
                     'to own a car','purchase of a car','to buy a car'] 
occasion_category = ['to have a wedding','having a wedding','wedding ceremony','purchase of my own house']
investment_category = ['housing transactions','buy real estate','buy commercial real estate',
                    'buy residential real estate','property','building a property',
                     'transactions with commercial real estate','building a real estate',
                       'housing','transactions with my real estate','real estate transactions',
                       'buying property for renting out']
Expenses_category = ['construction of own property','housing renovation',]
Education_category = ['going to university','getting higher education','university education',
                      'supplementary education','education','to become educated',
                       'getting an education','to get a supplementary education','profile education']

In [134]:
# Creating function for categorizing into different numerical groups based on ranges
def lemmatization_func(line):
    words = nltk.word_tokenize(line)
    lemmas = [wordnet_lemma.lemmatize(w, pos = "n") for w in words]
    lemmas=[l.lower() for l in lemmas]
    if any (word in lemmas for word in purchase_category):
        return "purchase"
    elif any (word in lemmas for word in occasion_category):
        return "occasion"
    elif any (word in lemmas for word in investment_category):
        return "investment"
    elif any (word in lemmas for word in Expenses_category):
        return "Expenses"
    elif any (word in lemmas for word in Education_category):
        return "Education"
    else:
        return "other"


In [135]:
# Creating column with categories
example=df.purpose[6]
example
lemmatization_func(example)

'housing transactions'

'investment'

In [136]:
df["purpose_loan"]=df['purpose'].apply(lemmatization_func)

In [137]:
df.purpose_loan.value_counts()

other         9594
investment    4441
purchase      4306
Education     3108
Name: purpose_loan, dtype: int64

In [138]:
# Count each categories values to see the distribution
len(Education_category)
len(Expenses_category)
len(investment_category)
len(occasion_category)
len(purchase_category)

9

2

12

4

11

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [139]:
# Check the children data and paying back on time
df.children.loc[(df.debt==1) & df.children==0].count()
df.children.loc[(df.debt==1) & df.children==1].count()
df.children.loc[(df.debt==1) & df.children==2].count()
df.children.loc[(df.debt==1) & df.children==3].count()
df.children.loc[(df.debt==1) & df.children==4].count()
df.children.loc[(df.debt==1) & df.children==5].count()
df.children.loc[(df.debt==0) & df.children==0].count()
df.children.loc[(df.debt==0) & df.children==1].count()
df.children.unique()

# Calculating default-rate based on the number of children



20977

472

0

0

0

0

16728

4721

array([1, 0, 3, 2, 4, 5])

<span style="color:blue; font-size: 18px">**accordingly it is just registered for families without children or with 1 child and the without they are the most back payer**

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 15 columns):
children            21449 non-null int64
days_employed       21449 non-null float64
dob_years           21449 non-null float64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null float64
purpose             21449 non-null object
age_group           21449 non-null object
max_days            21449 non-null float64
purpose_loan        21449 non-null object
dtypes: float64(4), int64(4), object(7)
memory usage: 3.2+ MB


**Conclusion**

[Write your conclusions based on your manipulations and observations.]


<span style="color:blue; font-size: 18px">**accordingly it shows that families with 0 or 1 children are the most borrowers and not paying back**

**Is there a correlation between family status and paying back on time?**

In [141]:
# Check the family status data and paying back on time
df.groupby(["family_status"])["debt"].mean().reset_index().sort_values(by="debt")
# Calculating default-rate based on family status

df.groupby(["children"])["debt"].mean().reset_index().sort_values(by="debt")

,family_status,debt
1,married,0.075470
2,unmarried,0.085012
0,civil partnership,0.093517


,children,debt
5,5,0.000000
0,0,0.075460
3,3,0.081818
1,1,0.091677
2,2,0.094925
4,4,0.097561


In [142]:
pd.pivot_table(df,index=["children","family_status"],
               values=["debt"],
               aggfunc=['count','sum','mean'])

count  sum      mean
                            debt debt      debt
children family_status                         
0        civil partnership  2728  229  0.083944
         married            7466  516  0.069113
         unmarried          3893  318  0.081685
1        civil partnership  1000  118  0.118000
         married            3003  247  0.082251
         unmarried           851   80  0.094007
2        civil partnership   355   33  0.092958
         married            1582  148  0.093552
         unmarried           191   21  0.109948
3        civil partnership    56    8  0.142857
         married             249   17  0.068273
         unmarried            25    2  0.080000
4        civil partnership     8    0  0.000000
         married              29    3  0.103448
         unmarried             4    1  0.250000
5        civil partnership     2    0  0.000000
         married               7    0  0.000000

**Conclusion**

[Write your conclusions based on your manipulations and observations.]

<span style="color:blue; font-size: 18px">**accordingly it shows that unmarried and with 4 children are the most borrowers and susceptible to not repaying credit**

In [143]:
pd.pivot_table(df,index=["family_status","children"],
               values=["debt"],
               aggfunc=['count','sum','mean'])

count  sum      mean
                            debt debt      debt
family_status     children                     
civil partnership 0         2728  229  0.083944
                  1         1000  118  0.118000
                  2          355   33  0.092958
                  3           56    8  0.142857
                  4            8    0  0.000000
                  5            2    0  0.000000
married           0         7466  516  0.069113
                  1         3003  247  0.082251
                  2         1582  148  0.093552
                  3          249   17  0.068273
                  4           29    3  0.103448
                  5            7    0  0.000000
unmarried         0         3893  318  0.081685
                  1          851   80  0.094007
                  2          191   21  0.109948
                  3           25    2  0.080000
                  4            4    1  0.250000

**Is there a correlation between income level and paying back on time?**

In [144]:
# Check the income level data and paying back on time
def level_income(income):
    A=(df['total_income'].max()-df['total_income'].min())/3
    A1=df['total_income'].max()-A
    A2=df['total_income'].max()-(2*A)
    if income <= A2:
        return 'low'
    if income <= A1:
        return 'middle'
    return 'high'
df["level_income"]=df['total_income'].apply(level_income)

# Calculating default-rate based on income level

df["level_income"].value_counts()

low       21404
middle       38
high          7
Name: level_income, dtype: int64

In [145]:
# Check the percentages for default rate for each credit purpose and analyze them
df.groupby(["level_income"])["debt"].mean().reset_index().sort_values(by="debt")

,level_income,debt
2,middle,0.078947
1,low,0.081153
0,high,0.142857


**Conclusion**

[Write your conclusions based on your manipulations and observations.]

<span style="color:blue; font-size: 18px">**accordingly it shows that high income level are the most borrowers and susceptible to not repaying credit**

**How does credit purpose affect the default rate?**

**Conclusion**

[Write your conclusions based on your manipulations and observations.]


In [146]:
#Reviewer's code 
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [147]:
df.groupby(["purpose_loan"])["debt"].mean().reset_index().sort_values(by="debt")

,purpose_loan,debt
1,investment,0.071831
2,other,0.076193
0,Education,0.092664
3,purchase,0.093590


<span style="color:blue; font-size: 18px">**accordingly it shows that purchase category has the most borrowers and susceptible to not repaying credit**

# General Conclusion 

[List your conclusions in this final section. Make sure you include all your important conclusions you made that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, and possible reasons and solutions for problematic artifacts that you had to address.]

[List your conclusions regarding the posed questions here as well.]


<span style="color:red; font-size: 18px">**General Conclusion**  
    <span style="color:blue; font-size: 15px">**I've learned:**  
        <span style="color:blue; font-size: 15px">**1. to have an analytical observation on data**  
        <span style="color:blue; font-size: 15px">**2. using the proper codes to check the missing values and their type.**  
        <span style="color:blue; font-size: 15px">**3. to build the steps and determine the goal that I have to achieve**  
        <span style="color:blue; font-size: 15px">**4. to start to transform the values and to fill in the missing values**  
        <span style="color:blue; font-size: 15px">**5. to check after every step if there any error had done.**  
        <span style="color:blue; font-size: 15px">**6. to start to build tables to get ideas about the default rates to gain some iformation about the relation between the different datas.**